In [2]:
from konlpy.tag import Mecab 

mec = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [3]:
import pandas as pd

df = pd.read_csv("df3_2_artsCleanEtc.csv")

# sample
df_sample= df.loc[:100]

In [19]:
mec.pos('공식문서입니다 학교에 다닙니다')

[('공식문서', 'NNP'),
 ('입니다', 'VCP+EF'),
 ('학교', 'NNG'),
 ('에', 'JKB'),
 ('다닙니다', 'VV+EF')]

In [5]:

series_mec = df_sample['art_title'].map(lambda x : mec.pos(x, flatten=False, join=False))

df_mec = pd.DataFrame(columns=['art_id','token'])

for i in range(len(series_mec)):
    for j in range(len(series_mec[i])):
        df_mec = df_mec.append({'art_id':i, 'token': series_mec[i][j]}, ignore_index=True)
        
# df_mec.head(len(df_mec))     
# df_mec.to_csv('df_mec_all.csv',encoding='utf-8-sig', index=False)


In [15]:
# (CASE1) 복합명사 처리x 기본 형태소 단위 그대로 넣은 경우 비교용

df_res_basic = pd.DataFrame(columns=['art_id','token','type'])

for i in range(len(df_mec)):
    for j in range(len(df_mec.loc[i][1])):
        df_res_basic = df_res_basic.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][j][0], 'type':df_mec.loc[i][1][j][1]},ignore_index=True)
               
print(df_res_basic)

df_res_basic.to_csv('df_res_basic.csv', encoding='utf-8-sig', index=False)

     art_id token    type
0         0   erp      SL
1         0  프로그램     NNG
2         0    오류     NNG
3         0    장애     NNG
4         0    조치     NNG
...     ...   ...     ...
1080    100     는      JX
1081    100   MAC      SL
1082    100     이     JKS
1083    100    정답     NNG
1084    100   일까요  VCP+EC

[1085 rows x 3 columns]


In [20]:
# (CASE4) mecab 배열상 type 연속성 기반 복합명사화 : NNG,NNP-NNG,NNP 혼재 CASE 
    # 문제: 3개 연달아 등장하는 명사 해결법
    # 문제: 연

df_res_NNG_NNP = pd.DataFrame(columns=['art_id','token','type'])
# df_res_NNG_NNP.append({'art_id':, 'token','type'})

for i in range(len(df_mec)):
    if len(df_mec.loc[i][1]) <= 1 :
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token':df_mec.loc[i][1][0][0],'type':df_mec.loc[i][1][0][1]},ignore_index=True)
        continue
    for j in range(1,len(df_mec.loc[i][1])):
        if (df_mec.loc[i][1][j-1][1] == 'NNG' or df_mec.loc[i][1][j-1][1] == 'NNP') and (df_mec.loc[i][1][j][1] == 'NNG' or df_mec.loc[i][1][j][1] == 'NNP') :
            df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][j-1][0], df_mec.loc[i][1][j][0] ]) ,'type':'comp_NNG_NNP'},ignore_index=True)
            continue
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][j][0] ,'type':df_mec.loc[i][1][j][1]}, ignore_index=True)


df_res_NNG_NNP.to_csv('df_res_NNG_NNP.csv', encoding='utf-8-sig', index=False)

print(df_res_NNG_NNP)

    art_id token          type
0        0   erp            SL
1        0  프로그램           NNG
2        0    오류           NNG
3        0  장애조치  comp_NNG_NNP
4        0    방법           NNG
..     ...   ...           ...
723    100     는           ETM
724    100   입니다        VCP+EC
725    100     는            JX
726    100     이           JKS
727    100   일까요        VCP+EC

[728 rows x 3 columns]
